<a href="https://colab.research.google.com/github/btlgs2000/Elettric80/blob/main/model_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
 # fashion mnist

# rete migliore come accuracy
# dropout rate (0, 0.5)
# numero di strati nascosti (1, 5)
# numero di unità per ogni strato nascosto (10, 200)
# relu e sigmoide

from tensorflow.keras import Model

In [4]:
def get_model(droput_rate:float, num_hidden:int, num_units:int, activation:str) -> Model:
    ''' Restituisce un modello '''

In [ ]:
def objective(trial):
    # scelta iperparametri
    # trial.suggest....

    # get_model

    # addestramento # 10 epoche con early stopping su accuratezza sul validation, patience=2

    # ritorna l'accuracy massima

In [ ]:
study = optuna.create_study()

In [ ]:
study.optimize()